In [ ]:
# Program finds the TMax from CHIRTSMax Data and a raster with polygon IDS burned
# By Cascade Tuholske 2019-08-20

#https://sebastianraschka.com/Articles/2014_multiprocessing.html

# Dependencies
import rasterio 
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from rasterio import features
import os
import xarray as xr
import fnmatch
import time

In [ ]:
# LOCAL TEST 
# CHIRT_DIR = '/Users/cascade/Github/UrbanHeat/data/test_in/' # <<--- path to loop through
# SHP_DIR = '/Users/cascade/Github/PopRaster/data/raw/JRC/ghs-ucdb/'
# POLY_RST_DIR = '/Users/cascade/Github/PopRaster/data/interim/'
# DATA_OUT = '/Users/cascade/Github/UrbanHeat/data/test_out/'

# TANA
# CHIRT_DIR = '/home/cascade/tana-spin-cascade/projects/CHIRTMax_Monthly/' # <<--- path to loop through
# SHP_DIR = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/raw/GHS_UCDB/'
# POLY_RST_DIR = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/interim/'
# DATA_OUT = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/processed'

# Tana Test
CHIRT_DIR = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_in/' # <<--- path to loop through
DATA_OUT = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_out/' # <<--- path to loop through
POLY_RST_DIR = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/interim/'
SHP_DIR = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/raw/GHS_UCDB/'

In [ ]:
# # Open Polygon Raster
# polyRst_fn = 'GHS_UCDB_Raster_Raster_touched.tif'
# polyRst = rasterio.open(POLY_RST_DIR+polyRst_fn)

# # Open the file with GeoPANDAS read_file
# shp_fn = 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp'
# shps = gpd.read_file(SHP_DIR+shp_fn)

# # Set fn out, change as needed 
# fn_out = 'GHS-CHIRT-MONTHLY'  

# # Isloate SHP Poly Col to merge back in later 
# df_ghs = gpd.GeoDataFrame()

# df_ghs['ID_HDC_G0'] = shps.ID_HDC_G0
# df_ghs['CTR_MN_NM'] = shps.CTR_MN_NM

In [ ]:
# Turn polyRst data as Xarray, 
# polyRst_da = xr.DataArray(polyRst.read(1), dims = ['y', 'x'])

In [37]:
# Loop through dirs in //

def temp_ghs(dir_nm):
    
    # print current process
    print(mp.current_process())
    
    #     print(type(dir_nm))

    # Open files 
    # Open Polygon Raster
    polyRst_fn = 'GHS_UCDB_Raster_Raster_touched.tif'
    polyRst = rasterio.open(POLY_RST_DIR+polyRst_fn)

    # Open the file with GeoPANDAS read_file
    shp_fn = 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp'
    shps = gpd.read_file(SHP_DIR+shp_fn)

    # Set fn out, change as needed 
    fn_out = 'GHS-CHIRT-MONTHLY'  

    # Isloate SHP Poly Col to merge back in later 
    df_ghs = gpd.GeoDataFrame()

    df_ghs['ID_HDC_G0'] = shps.ID_HDC_G0
    df_ghs['CTR_MN_NM'] = shps.CTR_MN_NM
    
    # Turn polyRst data as Xarray, 
    polyRst_da = xr.DataArray(polyRst.read(1), dims = ['y', 'x'])

    # Start Loop
    for fn in os.listdir(dir_nm):
        
        # Set dir name for writing files
        dir_year = dir_nm.split(CHIRT_DIR)[1]
        
        # find all the tif files
        if fn.endswith('.tif'):

                # NEED TO BUILD META DATA CHECK INTO ROUTINE and throw an error<<<<---------

                # Get the date of each chirt file
                date = (fn.split('CHIRTSmax.')[1].split('.tif')[0])
                print(dir_year)
                print(date)

                # Open CHIRT Data and turn data into array
                tempRst = rasterio.open(dir_nm+'/'+fn)

                # Make arrays into x    array DataArray
                tempRst_da = xr.DataArray(tempRst.read(1), dims = ['y', 'x']) # y and x are our 2-d labels

                # Make xarray dataset
                ds = xr.Dataset(data_vars = 
                        {'ghs' : (['y', 'x'], polyRst_da),
                        'temp' : (['y', 'x'], tempRst_da),})

                # UPDATED 2019-08-19 Mask the CHIRTS PIXELS FIRST, THEN GHS
                # Mask values from chirt that are ocean in ghs and chirt in our ds 
                ds_mask = ds.where(ds.temp != -9999, drop = False) #<<<<------ need to double check this

                # Mask pixels for both ghs and chirts where ghs cities are not present
                ds_mask = ds_mask.where(ds_mask.ghs > 0, drop = False)

                # Group poly_IDs find temp
                avg = ds_mask.groupby('ghs').mean(xr.ALL_DIMS)

                # turn GHS IDS and avg. CHIRTMax values into 1-D numpy arrays of equal length
                avg_ID = np.array(avg.ghs)
                avg_temp = np.array(avg.temp)

                print(len(avg_ID))
                print(len(avg_temp))

                # turn chirt max and IDS into a DF
                df_avg = pd.DataFrame()
                df_avg[date] = avg_temp
                df_avg['ID_HDC_G0'] = avg_ID

                # merge the df
                df_merge = df_ghs.merge(df_avg, on='ID_HDC_G0', how = 'outer') #<<<<----- NEED TO FIX THIS


    #df_merge.to_csv(DATA_OUT+fn_out+'_'+dir_year+'ParPro.csv') # csv out
    print('DONE ! ! !')

In [ ]:
# Test function 
def test_mp(dir_nm):
    # Start Loop
    
    # print current process
    print(mp.current_process())
    
    dir_year = dir_nm.split(CHIRT_DIR)[1].split('/')[0]

    fn_list = []
    for fn in os.listdir(dir_nm):
        
        # find all the tif files
        if fn.endswith('.tif'):

                # Get the date of each chirt file
                date = (fn.split('CHIRTSmax.')[1].split('.tif')[0])
                
                print(dir_year)
                print(date)
                
                fn_list.append(date)
    print(fn_list)
    # Save it out
    out_df = pd.DataFrame(fn_list)
    out_df.to_csv(DATA_OUT+dir_year+'.csv')

    #df_merge.to_csv(DATA_OUT+fn_out+'_'+dir_year+'ParPro.csv') # csv out
    print('DONE ! ! !')

# NEED TO FIX MERGE, SHOULD BE // otherwise whoot! Check clock speeds

In [34]:
# Get dir list
from glob import glob

dir_list= glob(CHIRT_DIR+'*/')
dir_list

['/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_in/Year1/',
 '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_in/Year2/',
 '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_in/Year3/',
 '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/test_in/Year4/']

In [38]:
from multiprocessing import Pool, Queue, Process
import time 
import os
import multiprocessing as mp

start = time.time()
 
pool = Pool(7)
pool.map(temp_ghs, dir_list)
#pool.map_async(temp_ghs, dir_list)
pool.close()

# for dir_nm in (dir_list):
#     proc = Process(target=temp_ghs, args=(dir_nm,))
#     proc.start()
#     proc.join()

end = time.time()
print(end-start)

<ForkProcess(ForkPoolWorker-77, started daemon)>
<ForkProcess(ForkPoolWorker-78, started daemon)>
<ForkProcess(ForkPoolWorker-79, started daemon)>
<ForkProcess(ForkPoolWorker-80, started daemon)>
Year3/
1983.06
Year1/
1983.01
Year4/
1983.10
Year2/
1983.08
13067
13067
Year1/
1983.02
13067
13067
Year4/
1983.11
13067
13067
Year3/
1983.04
13067
13067
Year2/
1983.07
13067
13067
13067
13067
Year1/
1983.03
Year4/
1983.12
13067
13067
Year3/
1983.05
13067
13067
Year2/
1983.09
13067
13067
DONE ! ! !
13067
13067
DONE ! ! !
13067
13067
DONE ! ! !
13067
13067
DONE ! ! !
71.409677028656
